# Planet Data Collection
Using the Open Exoplanet Catalogue database: https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/

## Data License
Copyright (C) 2012 Hanno Rein

Permission is hereby granted, free of charge, to any person obtaining a copy of this database and associated scripts (the "Database"), to deal in the Database without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Database, and to permit persons to whom the Database is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Database. A reference to the Database shall be included in all scientific publications that make use of the Database.

THE DATABASE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE DATABASE OR THE USE OR OTHER DEALINGS IN THE DATABASE.

## Follow instructions to get the xml file

In [1]:
import xml.etree.ElementTree as ET, urllib.request, gzip, io
url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems.xml.gz"
oec = ET.parse(gzip.GzipFile(fileobj=io.BytesIO(urllib.request.urlopen(url).read())))

## Parse into Pandas DataFrame
Information on what each field means can be found [here](https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/#data-structure).

In [66]:
import pandas as pd

def parse(base):
    db = oec.findall(f".//{base}")
    
    exclude = ['star', 'videolink', 'binary'] if base in ['system', 'binary'] else ['planet']
    
    columns = set([attribute.tag for attribute in db[0].getchildren() if attribute.tag not in exclude])
    results = pd.DataFrame(columns=columns)

    for entry in db:
        data = {col : entry.findtext(col) for col in columns}
        if base in ['system', 'binary']:
            data['binaries'] = len(entry.findall('.//binary'))
            data['stars'] = len(entry.findall('.//star'))
        if base in ['system', 'star', 'binary']:
            data['planets'] = len(entry.findall('.//planet'))
        results = results.append(data, ignore_index=True)

    return results

### Parse planet data

In [70]:
planets = parse('planet')
planets.head()

,period,name,eccentricity,description,discoverymethod,periastrontime,lastupdate,semimajoraxis,mass,periastron,list,discoveryyear
0,326.03,11 Com b,0.231,11 Com b is a brown dwarf-mass companion to th...,RV,2452899.6,15/09/20,1.29,19.4,94.8,Confirmed planets,2008
1,516.22,11 UMi b,0.08,11 Ursae Minoris is a star located in the cons...,RV,2452861.04,15/09/20,1.54,11.20,117.63,Confirmed planets,2009
2,185.84,14 And b,0,14 Andromedae is an evolved star in the conste...,RV,2452861.4,15/09/20,0.83,4.8,0,Confirmed planets,2008
3,1766,14 Her b,0.359,The star 14 Herculis is only 59 light years aw...,RV,None,15/09/21,2.864,4.975,22.230,Confirmed planets,2002
4,9886,14 Her c,0.184,14 Her c is the second companion in the system...,RV,None,15/09/21,9.037,7.679,189.076,Controversial,2006


### Parse system data

In [67]:
systems = parse('system')

In [68]:
systems.head()

,rightascension,name,distance,declination,binaries,planets,stars
0,12 20 43,11 Com,88.9,+17 47 34,0.0,1.0,1.0
1,15 17 05.88899,11 UMi,122.1,+71 49 26.0466,0.0,1.0,1.0
2,23 31 17.41346,14 And,79.2,+39 14 10.3092,0.0,1.0,1.0
3,16 10 23,14 Her,18.1,+43 49 18,0.0,2.0,1.0
4,19 41 48.95343,16 Cygni,21.146,+50 31 30.2153,2.0,1.0,3.0


### Parse binary data

In [57]:
binaries = parse('binary')
binaries.head()

,separation,name,positionangle,binaries,planets,stars
0,39.56,16 Cygni,133.30,1.0,1.0,3.0
1,3.4,16 Cygni AC,209,0.0,0.0,2.0
2,12.37,2M0441+2301,237.3,1.0,1.0,3.0
3,0.2323,2M 044145,79.61,0.0,0.0,2.0
4,None,2M 1938+4603,None,0.0,1.0,2.0


### Parse star data

In [69]:
stars = parse('star')
stars.head()

,magJ,name,radius,spectraltype,magB,magV,temperature,magK,mass,metallicity,magH,planets
0,2.943,11 Com,19,G8 III,5.74,4.74,4742,2.282,2.7,-0.35,2.484,1.0
1,2.876,11 UMi,24.08,K4III,6.415,5.024,4340,1.939,1.80,0.04,2.091,1.0
2,3.019,14 And,11,K0III,6.24,5.22,4813,2.331,2.2,-0.24,2.608,1.0
3,5.158,14 Her,0.708,K0 V,7.57,6.67,5311.0,4.714,1.0,0.43,4.803,2.0
4,5.09,16 Cygni A,1.243,G2V,6.59,5.95,5825,4.43,1.11,0.096,4.72,0.0


## Save to CSVs

In [71]:
planets.to_csv('data/planets.csv', index=False)
binaries.to_csv('data/binaries.csv', index=False)
stars.to_csv('data/stars.csv', index=False)
systems.to_csv('data/systems.csv', index=False)